In [7]:
import SimpleITK as sitk
import os

### SimpleElastix Registration

In [6]:
# Concatenate the ND images into one (N+1)D image
# population = ['image1.hdr', ..., 'imageN.hdr']
path = 'E:/drbrain/StrokeAI/data/PWI/hdr'
population = os.listdir(path)
print(population)
    
vectorOfImages = sitk.VectorOfImage()

for filename in population:
    if '.hdr' in filename:
        print(filename)
        vectorOfImages.push_back(sitk.ReadImage(os.path.join(path, filename)))

image = sitk.JoinSeries(vectorOfImages)
print(image)
# Register
elastixImageFilter = sitk.SimpleElastix() #sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(image)
elastixImageFilter.SetMovingImage(image)
elastixImageFilter.SetParameterMap(sitk.GetDefaultParameterMap('groupwise'))
elastixImageFilter.Execute()

### sitk registration

In [9]:
from __future__ import print_function
from functools import reduce


import SimpleITK as sitk
# import sys
import os


def command_iteration(method) :
    print("{0:3} = {1:10.5f} : {2}".format(method.GetOptimizerIteration(),
                                           method.GetMetricValue(),
                                           method.GetOptimizerPosition()))



# if len ( sys.argv ) < 4:
#     print( "Usage: {0} <fixedImageFilter> <movingImageFile>  <outputTransformFile>".format(sys.argv[0]))
#     sys.exit ( 1 )
    
    
fixedFile= 'E:/drbrain/StrokeAI/data/PWI/new/13/1_0.dcm'
movingFile= 'E:/drbrain/StrokeAI/data/PWI/new/13/2_1.dcm'
outputTransformFile= 'E:/drbrain/StrokeAI/data/new/reg/13.tfm'

pixelType = sitk.sitkFloat32

fixed3d = sitk.ReadImage(fixedFile, sitk.sitkFloat32)
fixed = sitk.Extract(fixed3d, (fixed3d.GetWidth(), fixed3d.GetHeight(), 0), (0, 0, 0))
# fixed = sitk.Normalize(fixed)
# fixed = sitk.DiscreteGaussian(fixed, 2.0)

moving3d = sitk.ReadImage(movingFile, sitk.sitkFloat32)
moving = sitk.Extract(moving3d, (moving3d.GetWidth(), moving3d.GetHeight(), 0), (0, 0, 0))
# moving = sitk.Normalize(moving)
# moving = sitk.DiscreteGaussian(moving, 2.0)

numberOfBins = 24
samplingPercentage = 0.10

R = sitk.ImageRegistrationMethod()

R.SetMetricAsMattesMutualInformation(numberOfBins)
R.SetMetricSamplingPercentage(samplingPercentage,sitk.sitkWallClock)
R.SetMetricSamplingStrategy(R.RANDOM)
R.SetOptimizerAsRegularStepGradientDescent(1.0,.001,200)

R.SetInitialTransform(sitk.TranslationTransform(fixed.GetDimension()))
R.SetInterpolator(sitk.sitkLinear)

R.AddCommand( sitk.sitkIterationEvent, lambda: command_iteration(R) )

outTx = R.Execute(fixed, moving)

print("-------")
print(outTx)
print("Optimizer stop condition: {0}".format(R.GetOptimizerStopConditionDescription()))
print(" Iteration: {0}".format(R.GetOptimizerIteration()))
print(" Metric value: {0}".format(R.GetMetricValue()))


sitk.WriteTransform(outTx,  outputTransformFile)

if ( not "SITK_NOSHOW" in os.environ ):

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed);
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(100)
    resampler.SetTransform(outTx)

    out = resampler.Execute(moving)

    simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
    simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
    cimg = sitk.Compose(simg1, simg2, simg1//2.+simg2//2.)
    sitk.WriteImage(cimg, 'E:/drbrain/StrokeAI/data/new/reg/13.nii')
#     sitk.Show( cimg, "ImageRegistration2 Composition" )

  0 =   -0.29042 : (-0.08600094017304871, 0.996295055839058)
  1 =   -0.29506 : (0.896347954924912, 1.183352930037969)
  2 =   -0.29439 : (0.4036738381137956, 1.0980756492160149)
  3 =   -0.29673 : (-0.08608706944018296, 1.1987446783754503)
  4 =   -0.29533 : (0.09953901434416332, 1.0312843636173357)
  5 =   -0.29511 : (-0.002388454614219812, 1.1036431299952794)
  6 =   -0.29513 : (-0.12436519262439086, 1.130968491555524)
  7 =   -0.29535 : (-0.06188045659097259, 1.129587273674797)
  8 =   -0.29526 : (-0.024742828478798065, 1.0793175354797175)
  9 =   -0.29514 : (-0.055070878735631455, 1.086852246331435)
 10 =   -0.29520 : (-0.03959594723436652, 1.0846918945010244)
 11 =   -0.29517 : (-0.047249792343372427, 1.0862583590864352)
-------
itk::simple::Transform
 TranslationTransform (0000018EACA52FA0)
   RTTI typeinfo:   class itk::TranslationTransform<double,2>
   Reference Count: 2
   Modified Time: 8850
   Debug: Off
   Object Name: 
   Observers: 
     none
   Offset: [-0.0472498, 1.08

#### script version

In [ ]:
'''

#!/usr/bin/env python

from __future__ import print_function
from functools import reduce


import SimpleITK as sitk
import sys
import os


def command_iteration(method) :
    print("{0:3} = {1:7.5f} : {2}".format(method.GetOptimizerIteration(),
                                           method.GetMetricValue(),
                                           method.GetOptimizerPosition()))



if len ( sys.argv ) < 4:
    print( "Usage: {0} <fixedImageFilter> <movingImageFile>  <outputTransformFile>".format(sys.argv[0]))
    sys.exit ( 1 )

pixelType = sitk.sitkFloat32

fixed = sitk.ReadImage(sys.argv[1], sitk.sitkFloat32)
fixed = sitk.Normalize(fixed)
fixed = sitk.DiscreteGaussian(fixed, 2.0)


moving = sitk.ReadImage(sys.argv[2], sitk.sitkFloat32)
moving = sitk.Normalize(moving)
moving = sitk.DiscreteGaussian(moving, 2.0)


R = sitk.ImageRegistrationMethod()

R.SetMetricAsJointHistogramMutualInformation()

R.SetOptimizerAsGradientDescentLineSearch(learningRate=1.0,
                                          numberOfIterations=200,
                                          convergenceMinimumValue=1e-5,
                                          convergenceWindowSize=5)

R.SetInitialTransform(sitk.TranslationTransform(fixed.GetDimension()))

R.SetInterpolator(sitk.sitkLinear)

R.AddCommand( sitk.sitkIterationEvent, lambda: command_iteration(R) )

outTx = R.Execute(fixed, moving)

print("-------")
print(outTx)
print("Optimizer stop condition: {0}".format(R.GetOptimizerStopConditionDescription()))
print(" Iteration: {0}".format(R.GetOptimizerIteration()))
print(" Metric value: {0}".format(R.GetMetricValue()))


sitk.WriteTransform(outTx,  sys.argv[3])

if ( not "SITK_NOSHOW" in os.environ ):

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed);
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(1)
    resampler.SetTransform(outTx)

    out = resampler.Execute(moving)

    simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
    simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
    cimg = sitk.Compose(simg1, simg2, simg1//2.+simg2//2.)
    sitk.Show( cimg, "ImageRegistration2 Composition" )
    
    
'''